# 얼굴인식 잠금해제

### Step1 - 얼굴 검출하여 얼굴 사진 수집하기 100장

In [2]:
# 하르 캐스케이드 얼굴 검출 (Haar Cascade Face Detection) 사용 : cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
import cv2
import numpy as np
import os

if not os.path.exists('faces'):
    os.makedirs('faces')
    
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(0)
count = 0

while(True):
    ret, frame = camera.read()                              # 프레임 단위로 캡처
    
    gray_camera = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_camera, 1.3, 6)

    COLORG = (0, 200, 2) # 녹색
    COLORB = (200, 2 ,0)
    if faces is not() :
        count += 1
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+ h), COLORB, 2)
            text = "ANALYZING..."
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
            text_x = int((x + x + w - text_size[0]) / 2)
            text_y = int(y + h + text_size[1] + 10)  # 상자 아래에 글자 표시
            cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            crop_face = gray_camera[y:y+h, x:x+w]
        face = cv2.resize(crop_face, (200,200))
        # faces/user1.jpg, faces/user2.jpg ...
        file_name_path = os.path.join('faces', f'user{count}.jpg')
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        cv2.imshow('face crop', face)
    else : 
        pass
    
    cv2.imshow('Press Spacebar to Exit',frame)              # 프레임 표시
    
    if cv2.waitKey(1) & 0xFF == ord(' ') or count == 100:  # 스페이스바가 감지되면 중지
        break

camera.release()                           # 스페이스바가 감지된 후 창을 종료
cv2.destroyAllWindows()
print('Analysis is success')

<>:21: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:21: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\isfs0\AppData\Local\Temp\ipykernel_14620\989535963.py:21: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if faces is not() :


Analysis is success


### Step2 - 100장의 사진을 학습 시키기
 
   모델 학습 전 i를 32비트로 변화하는 이유 : 모델 학습의 원활한 진행과 정확도 향상을 위한 중요한 단계 중 하나
   - 데이터 형식 표준화: 머신 러닝 모델 학습에는 입력 데이터와 레이블 데이터의 데이터 형식이 일치해야 합니다. 모델 학습 시에 데이터 형식의 불일치로 인한 문제를 방지하기 위해 레이블 데이터를 표준 데이터 형식인 32비트로 변환합니다.
   - 최적화 및 호환성: 일반적으로 많은 머신 러닝 프레임워크와 라이브러리는 32비트 정수(np.int32)를 레이블 데이터의 기본 형식으로 사용합니다. 따라서 32비트로 변환하면 호환성 문제가 줄어들고, 모델 학습 및 예측 과정이 최적화될 수 있습니다.
   - 정수 레이블: 레이블 데이터가 정수로 표현되는 경우가 많습니다. 예를 들어, 얼굴 인식 모델의 경우 얼굴을 식별하는 데 사용되는 레이블은 일반적으로 숫자로 표현됩니다. 따라서 레이블을 정수 형식으로 변환하여 모델이 해당 레이블을 이해하고 처리할 수 있도록 합니다.
   - 계산 및 메모리 효율성: 32비트 정수는 대부분의 시스템에서 효율적으로 처리할 수 있으며, 메모리 사용량이 적습니다. 이는 대규모 데이터셋 및 모델 학습 시에 중요한 역할을 합니다.

In [ ]:
%pip install opencv-python-headless==4.8.1.78 --user
%pip install opencv-contrib-python==4.8.1.78 --user

In [ ]:
!python -m pip install --upgrade pip
!pip3 install scikit-build
!pip install cmake
!pip install --user opencv-contrib-python==4.6.0.66
!pip install opencv-python-headless==4.8.1.78
!pip install opencv-contrib-python==4.8.1.78

In [4]:
# model = cv2.face.LBPHFaceRecognizer_create()
# model.train(.....)

import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

file_path = 'faces/'
# 폴더 안에 파일 리스트로 변환
onlyfiles = [f for f in listdir(file_path) if isfile(join(file_path, f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles) :
    image_path = file_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if images is None :
        continue
    # Training_Data 리스트에 바이트 배열로 이미지 추가
    # 바이트 배열로 변환하면 데이터베이스에 저장하거나 네트워크를 통해 전송에 유용
    # np.uint8 == 8비트 부호 없는 정수 형식
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

if len(Labels) == 0 :
    print("NONDATA")
    exit()

Labels = np.asarray(Labels, dtype=np.int32)

model = cv2.face.LBPHFaceRecognizer_create()

model.train(np.asarray(Training_Data), np.asarray(Labels))
model.write('./faces/my_face.xml')
cv2.imshow("First face", Training_Data[0])
cv2.imshow("Last face", Training_Data[99])
cv2.waitKey(0)
cv2.destroyAllWindows()
print("Model training complete")

Model training complete


### Step3 - 얼굴 인식해서 학습한 인물인지 확인하여 'Access Granted' 나타내고  & 정확도 표 시

In [5]:
import cv2
import numpy as np
import os, glob

file_path = 'faces/'

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
model = cv2.face.LBPHFaceRecognizer_create()
model.read(os.path.join(file_path,'my_face.xml'))

dirs = [d for d in glob.glob(file_path+"/*") if os.path.isdir(d)]
print(dirs)

COLORG = (0, 200, 2) # 녹색
COLORR = (2, 0, 200)
COLORB = (200, 2, 0)

ANALYZING_TIMEOUT = 3

camera = cv2.VideoCapture(0)

while camera.isOpened() :
    ret, frame = camera.read()
    if not ret : 
        print("no frame")
        break

    gray_camera = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_camera, 1.3, 6)
    
    for (x,y,w,h) in faces : 
        cv2.rectangle(frame, (x, y), (x+w, y+ h), COLORB, 2)
        # text = "ANALYZING..."
        # text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        # text_x = int((x + x + w - text_size[0]) / 2)
        # text_y = int(y + h + text_size[1] + 10)  # 상자 아래에 글자 표시
        # cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (200,200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        labels, confidence = model.predict(face)

        if confidence < 500 :
            accuracy = int(100*(1-confidence/300))
       
            if accuracy > 80 :
                cv2.rectangle(frame, (x, y), (x+w, y+ h), COLORG, 2)
                text = "ACCESS SUCCESS"
                color = COLORG
            
            else :
                cv2.rectangle(frame, (x, y), (x+w, y+ h), COLORR, 2)
                text = "ACCESS DENIED"
                color = COLORR
                
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
            text_x = int((x + x + w - text_size[0]) / 2)
            text_y = int(y + h + text_size[1] + 10)  # 상자 아래에 글자 표시
            cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.putText(frame, f'{accuracy}%', (text_x, text_y + text_size[1] + 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow('Press Spacebar to Exit',frame)              # 프레임 표시
    
    if cv2.waitKey(1) & 0xFF == ord(' ') :  # 스페이스바가 감지되면 중지
        break
camera.release()                           # 스페이스바가 감지된 후 창을 종료
cv2.destroyAllWindows()
    

[]


In [2]:
import cv2
import numpy as np

def collect_face_images():
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)
    
    face_samples = []  # 얼굴 이미지를 저장할 리스트
    count = 0  # 얼굴 이미지 수
    
    while True:
        ret, frame = cap.read()  # 웹캠에서 프레임 읽기
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 흑백으로 변환
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)  # 얼굴 검출
        # multiscale 은 여러명의 얼굴이 뜰 수 있으니 내 얼굴을 기준으로 x,y 좌표를 잡으라는 정보값 지정 
    
        for (x, y, w, h) in faces:
            face = gray[y:y + h, x:x + w]
            face_samples.append(face)
            count += 1
    
            if count >= 100:
                break
    
        cv2.imshow('Collecting Faces', frame)
    
        if count >= 100:
            break
    
        if cv2.waitKey(1) == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    return face_samples

In [6]:
# Step 2: 얼굴 이미지를 학습시키기
def train_face_recognition(face_samples):
    model = cv2.face_LBPHFaceRecognizer.create()
    labels = np.array([1] * len(face_samples))  # 레이블은 모두 1로 설정 (Access Granted)
    
    model.train(face_samples, labels)
    
    return model



In [8]:
# 웹캠 다시 시작하여 얼굴 인식 수행 (Step 3)
def face_recognition_with_model(model):
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
        for (x, y, w, h) in faces:
            face = gray[y:y + h, x:x + w]
            label, confidence = model.predict(face)
    
            if label == 1:  # 예측 레이블 확인하여 인물 확인
                cv2.putText(frame, "Access Granted", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.putText(frame, f"Confidence: {confidence}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Access Denied", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                cv2.putText(frame, f"Confidence: {confidence}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    
        cv2.imshow('Face Recognition', frame)
    
        if cv2.waitKey(1) == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# # Step 1: 얼굴 이미지 수집
# face_samples = collect_face_images()

# # Step 2: 학습
# model = train_face_recognition(face_samples)

# # Step 3: 얼굴 인식
# face_recognition_with_model(model)

# 팀원 얼굴 인식

In [22]:
import cv2
import numpy as np

def collect_face_images(output_folder, max_samples=100):
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    camera = cv2.VideoCapture(0)
    
    face_samples = []  # 얼굴 이미지를 저장할 리스트
    sample_count = 0   # 수집한 샘플 수
    
    while True:
        ret, frame = camera.read()
        if not ret:
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
        
        for (x, y, w, h) in faces:
            face = gray_frame[y:y+h, x:x+w]
            face_samples.append(face)
            sample_count += 1
            
            # 샘플 이미지를 파일로 저장 (예: sample_001.jpg, sample_002.jpg, ...)
            file_name = f"{output_folder}/sample_{sample_count:03d}.jpg"
            cv2.imwrite(file_name, face)
        
        cv2.imshow('Collecting Faces', frame)
        
        if sample_count >= max_samples:
            break
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    camera.release()
    cv2.destroyAllWindows()
    return face_samples

# 샘플 얼굴 이미지를 저장할 폴더 지정
output_folder = "collected_samples"

# Step 1 실행: 얼굴 이미지 수집
collected_face_samples = collect_face_images(output_folder, max_samples=100)

print("Step 1: Face image collection completed.")

Step 1: Face image collection completed.


In [23]:
def train_face_recognition(face_samples):
    model = cv2.face_LBPHFaceRecognizer.create()
    labels = np.array([1] * len(face_samples))  # 레이블은 모두 1로 설정 (Access Granted)
    
    model.train(face_samples, labels)
    return model

# Step 2 실행: 얼굴 인식 모델 학습
trained_model = train_face_recognition(collected_face_samples)

# print("Step 2: Face recognition model trained.")

In [24]:
def face_recognition_with_model(model):
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    camera = cv2.VideoCapture(0)
    
    while True:
        ret, frame = camera.read()
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
        for (x, y, w, h) in faces:
            face = gray_frame[y:y + h, x:x + w]
            label, confidence = model.predict(face)
    
            if label == 1:  # 예측 레이블 확인하여 인물 확인 (Access Granted)
                cv2.putText(frame, "Access Granted", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.putText(frame, f"Confidence: {confidence:.2f}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Access Denied", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                cv2.putText(frame, f"Confidence: {confidence:.2f}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    
        cv2.imshow('Face Recognition', frame)
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    camera.release()
    cv2.destroyAllWindows()

# # Step 3 실행: 얼굴 인식
# face_recognition_with_model(trained_model)

# print("Step 3: Face recognition completed.")